In [1]:
!pip install -q stellargraph[demos]==1.2.1

In [2]:
!pip install -q openpyxl

In [5]:
# !pip install tensorflow-gpu

In [6]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.2.1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.2.1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

In [7]:
!ls

dummy.xlsx	      GCN-3 networkx for tf-pathway.ipynb
file_for_network.csv  GCN-4 stelllargraph.ipynb
foo3.pdf	      GCN.ipynb
foo3.png	      pathway-tf.xlsx
foo.pdf		      tfc_graph_analysis.csv
foo.png		      tfc_graph_colour.png
GCN-2.ipynb


In [13]:
from stellargraph import StellarGraph
import json
import pandas as pd
import numpy as np
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.metrics import mean_absolute_error, mean_squared_error

import stellargraph as sg
from stellargraph.mapper import HinSAGELinkGenerator
from stellargraph.layer import HinSAGE, link_regression
from tensorflow.keras import Model, optimizers, losses, metrics

import multiprocessing
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
import pandas as pd

In [15]:
xl = pd.ExcelFile("pathway-tf.xlsx")
df = xl.parse("Sheet1", header=None)
df.columns=["pathway","tfs"]
# df=df.head()
df

,pathway,tfs
0,acute myeloid leukemia,"['STAT3', 'PRDM1']"
1,adipocytokine signaling pathway,"['STAT3', 'PRDM1']"
2,pancreatic cancer,"['STAT3', 'PRDM1']"
3,jak stat signaling pathway,"['STAT3', 'PRDM1']"
4,acute myeloid leukemia,"['STAT3', 'KLF5']"
...,...,...
1048570,cell cycle,"['LEF1', 'MYC', 'SNAI1', 'NANOG', 'POU5F1', 'S..."
1048571,focal adhesion,"['LEF1', 'MYC', 'SNAI1', 'NANOG', 'POU5F1', 'S..."
1048572,neuroactive ligand receptor interaction,"['LEF1', 'MYC', 'SNAI1', 'NANOG', 'POU5F1', 'S..."
1048573,prostate cancer,"['SMAD4', 'TBX18', 'SNAI2', 'STAT1', 'TBX21', ..."


In [16]:
df.tfs[1].__class__

str

In [17]:
# import ast
# for i in range(len(df)):
#     x =df.tfs[i]
#     x = ast.literal_eval(x)
#     x = [n.strip() for n in x]
#     df.tfs[i]=x


In [18]:
# x.__class__

In [19]:
# df[['tf1','tf2']] = pd.DataFrame(df.tfs.tolist(), index= df.index)

In [20]:
# df.drop(["tfs"],axis=1, inplace=True)
# df

In [21]:
print(df.shape)
df.head()

(1048575, 2)


,pathway,tfs
0,acute myeloid leukemia,"['STAT3', 'PRDM1']"
1,adipocytokine signaling pathway,"['STAT3', 'PRDM1']"
2,pancreatic cancer,"['STAT3', 'PRDM1']"
3,jak stat signaling pathway,"['STAT3', 'PRDM1']"
4,acute myeloid leukemia,"['STAT3', 'KLF5']"


In [22]:
df=df.drop_duplicates(subset=['pathway', 'tfs'], keep=False)
df.shape

(42918, 2)

In [32]:
df.columns=["target","source"]

## Stellargraph

In [33]:
square_edges = df


,target,source
24,lysine degradation,"['SETDB1', 'STAT3']"
29,methionine metabolism,"['STAT3', 'DNMT1']"
174,vegf signaling pathway,"['STAT3', 'NFATC2']"
177,axon guidance,"['STAT3', 'NFATC2']"
178,natural killer cell mediated cytotoxicity,"['STAT3', 'NFATC2']"
...,...,...
1048493,notch signaling pathway,"['HMGA2', 'SOX2', 'NANOG', 'POU5F1', 'SALL4', ..."
1048552,vegf signaling pathway,"['LEF1', 'MYC', 'SNAI1', 'NANOG', 'POU5F1', 'S..."
1048556,circadian rhythm,"['LEF1', 'MYC', 'SNAI1', 'NANOG', 'POU5F1', 'S..."
1048557,axon guidance,"['LEF1', 'MYC', 'SNAI1', 'NANOG', 'POU5F1', 'S..."


In [37]:
square_named = StellarGraph(
    edges=square_edges, node_type_default="tfs", edge_type_default="pathway"
)
print(square_named.info())

StellarGraph: Undirected multigraph
 Nodes: 9784, Edges: 42918

 Node types:
  tfs: [9784]
    Features: none
    Edge types: tfs-pathway->tfs

 Edge types:
    tfs-pathway->tfs: [42918]
        Weights: all 1 (default)
        Features: none


In [39]:
square_named

In [41]:
square_edges_first_second = square_edges.rename(
    columns={"source": "tfs", "target": "pathway"}
)
square_edges_first_second

,pathway,tfs
24,lysine degradation,"['SETDB1', 'STAT3']"
29,methionine metabolism,"['STAT3', 'DNMT1']"
174,vegf signaling pathway,"['STAT3', 'NFATC2']"
177,axon guidance,"['STAT3', 'NFATC2']"
178,natural killer cell mediated cytotoxicity,"['STAT3', 'NFATC2']"
...,...,...
1048493,notch signaling pathway,"['HMGA2', 'SOX2', 'NANOG', 'POU5F1', 'SALL4', ..."
1048552,vegf signaling pathway,"['LEF1', 'MYC', 'SNAI1', 'NANOG', 'POU5F1', 'S..."
1048556,circadian rhythm,"['LEF1', 'MYC', 'SNAI1', 'NANOG', 'POU5F1', 'S..."
1048557,axon guidance,"['LEF1', 'MYC', 'SNAI1', 'NANOG', 'POU5F1', 'S..."


In [43]:
square_first_second = StellarGraph(
    edges=square_edges_first_second, source_column="tfs", target_column="pathway",node_type_default="tfs", edge_type_default="pathway"
)
print(square_first_second.info())

StellarGraph: Undirected multigraph
 Nodes: 9784, Edges: 42918

 Node types:
  tfs: [9784]
    Features: none
    Edge types: tfs-pathway->tfs

 Edge types:
    tfs-pathway->tfs: [42918]
        Weights: all 1 (default)
        Features: none


In [45]:

import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, HinSAGE, link_classification

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
%matplotlib inline

In [47]:
G=square_first_second
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 9784, Edges: 42918

 Node types:
  tfs: [9784]
    Features: none
    Edge types: tfs-pathway->tfs

 Edge types:
    tfs-pathway->tfs: [42918]
        Weights: all 1 (default)
        Features: none


In [48]:
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=True
)

** Sampled 4291 positive and 4291 negative edges. **


In [51]:
# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(G_test)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=True
)

** Sampled 3862 positive and 3862 negative edges. **


In [52]:
print(G_train.info())

StellarGraph: Undirected multigraph
 Nodes: 9784, Edges: 34765

 Node types:
  tfs: [9784]
    Features: none
    Edge types: tfs-pathway->tfs

 Edge types:
    tfs-pathway->tfs: [34765]
        Weights: all 1 (default)
        Features: none


In [50]:
print(G_test.info())

StellarGraph: Undirected multigraph
 Nodes: 9784, Edges: 38627

 Node types:
  tfs: [9784]
    Features: none
    Edge types: tfs-pathway->tfs

 Edge types:
    tfs-pathway->tfs: [38627]
        Weights: all 1 (default)
        Features: none


In [53]:
batch_size = 20
epochs = 20

In [54]:
num_samples = [20, 10]

In [ ]:
train_gen = GraphSAGELinkGenerator(G_train, batch_size, num_samples)
train_flow = train_gen.flow(edge_ids_train, edge_labels_train, shuffle=True)

In [55]:
test_gen = GraphSAGELinkGenerator(G_test, batch_size, num_samples)
test_flow = test_gen.flow(edge_ids_test, edge_labels_test)

RuntimeError: This StellarGraph has no numeric feature attributes for nodesNode features are required for machine learning